```haskell
<*> :: f (a -> b) -> a -> b

>>= :: m a -> (a -> m b) -> m b

In [1]:
(<*>) :: Monad m => m (a -> b) -> m a -> m b
mf <*> mx = mf >>= \f -> mx >>= \x -> return (f x)

In [8]:
fmap :: Monad f => (a -> b) -> f a -> f b
fmap f x = pure f <*> x

Line 2: Use <$>
Found:
pure f <*> x
Why not:
f <$> x

In [73]:
(<$) :: (MyMonad f, MyFunctor f, MyApplicative f) => a -> f b -> f a
-- x <$ fb = fmap (const x) fb
-- x <$ fb = return x
x <$ fb = return (const x) <*> fb

Line 4: Use <$>
Found:
return (const x) <*> fb
Why not:
const x <$> fb

In [64]:
(*>) :: MyMonad m => m a -> m b -> m b
ma *> mb = mb >>= \b -> return b



In [12]:
fmap :: Monad m => (a -> b) -> m a -> m b
fmap f m = m >>= \a -> return $ f a

In [14]:
data Tree a = Leaf a | Node (Tree a) a (Tree a)

In [28]:
instance Functor Tree where
    fmap :: (a -> b) -> Tree a -> Tree b
    f `fmap` Leaf a = Leaf $ f a
    f `fmap` Node l c r = Node (fmap f l) (f c) (fmap f r)

In [34]:
instance Applicative Tree where
    pure :: a -> Tree a
    pure x = Leaf x

    (<*>) :: Tree (a -> b) -> Tree a -> Tree b
    Leaf f <*> Node l c r = Node (Leaf f <*> l) (f c) (Leaf f <*> r)


    (*>) :: Tree a -> Tree b -> Tree b
    ta *> tb = tb

    (<*) :: Tree a -> Tree b -> Tree a
    ta <* tb = ta

Line 3: Eta reduce
Found:
pure x = Leaf x
Why not:
pure = Leaf

In [26]:
instance Monad Tree where
    return :: a -> Tree a
    return = Leaf

    (>>=) :: Tree a -> (a -> Tree b) -> Tree b
    Leaf x >>= f = f x
    Node l c r >>= f = Node (l >>= f) tc (r >>=f) where Leaf tc = f c


In [38]:
foldTree :: (a -> b) -> (b -> a -> b) -> Tree a -> b
foldTree fl _ (Leaf a) = fl a

In [48]:
class MyMonad m where
    (>>=) :: m a -> (a -> m b) -> m b
    return :: a -> m a

class MyApplicative f where
    pure :: a -> f a
    (<*>) :: f (a -> b) -> f a -> f b

class MyFunctor f where
    fmap :: (a -> b) -> f a -> f b
    

In [40]:
data Rt a = Rt a [Rt a]

In [49]:
instance MyFunctor Rt where
    fmap :: (a -> b) -> Rt a -> Rt b
    f `fmap` Rt a [] = Rt (f a) []
    f `fmap` Rt a as = Rt (f a) [fmap f b | b <- as]

In [56]:
instance MyApplicative Rt where
    (<*>) :: Rt (a -> b) -> Rt a -> Rt b
    Rt f [] <*> Rt a as = Rt (f a) [fmap f b | b <- as]
    Rt f fs <*> Rt a as = Rt (f a) [g <*> b | b <- as, g <- fs]

In [61]:
instance MyMonad Rt where
    return :: a -> Rt a
    return x = Rt x []

    (>>=) :: Rt a -> (a -> Rt b) -> Rt b
    Rt a as >>= f = Rt m [b >>= f | b <- as] where Rt m [] = f a
